<a href="https://colab.research.google.com/github/Iriszhaiq/498CA_Final_Proj/blob/master/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:

downloaded = drive.CreateFile({'id':'1Db4GUynKccPaL0cPEdMil0ff8sM7HplZ'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('Reputation_prediction_data_train.txt') # now you can use txt file

downloaded2 = drive.CreateFile({'id':'1wAymDbcre5f2ySAXas-f-wsc47F0ELXK'}) # replace fileid with Id of file you want to access
downloaded2.GetContentFile('Reputation_prediction_labels_train.txt') # now you can use txt file

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn import metrics
import pandas as pd
import numpy as np
from pandas import DataFrame 
import os

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
#===================================== read datas =====================================
user_id,content_type,timestamp,textual=[],[],[],[]
with open('Reputation_prediction_data_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[-3])
        content_type.append(data[-2])
        timestamp.append(data[-1])
        textual.append(data[:-3])
df_data = DataFrame({'user_id': user_id})
df_data['user_id']=user_id
df_data['content_type'] = content_type
df_data['textual'] = textual
#===================================== read labels =====================================
user_id, label = [],[]
with open('Reputation_prediction_labels_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[0])
        label.append(data[1])
df_test = DataFrame()
df_test['label']=label
df_test['user_id'] = user_id
#===================================== merge 2 dataframe =====================================
df = pd.merge(df_data, df_test, on='user_id',how='left')
#===================================== group data =====================================
df_group = df.groupby(['user_id','content_type','label']).agg({'textual':'sum'}).add_suffix('_comb').reset_index()
df_group['textual_comb']=[' '.join(i) for i in df_group.textual_comb]
df_group.shape  #(6535, 4)

(6535, 4)

In [0]:
df[190:200]

,user_id,content_type,textual,label
190,13,Q,"[basic, premis, restorationist, movement, ear,...",1
191,13,Q,"[distinct, ordin, type, lds, church, recent, l...",1
192,13,Q,"[origin, bridal, veil, christian, wed, practic...",1
193,13,Q,"[accord, kingdom, theolog, man, hold, secular,...",1
194,13,Q,"[differ, elder, led, elder, rule, church, gove...",1
195,13,Q,"[distinct, exhibit, true, furqan, book, true, ...",1
196,13,Q,"[matthew, mark, luke, call, synopt, gospel, go...",1
197,14,Q,"[fast, daniel, cyrus, king, persia, reveal, da...",3
198,14,Q,"[pray, bold, pray, person, achiev, pray, selfl...",3
199,15,Q,"[differ, judg, god, question, god, answer, man...",1


In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_group.textual_comb,df_group.label, test_size=0.2,random_state=6)
train, test = train_test_split(df_group, test_size=0.2，random_state=6)
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                     ('clf', MultinomialNB(alpha = 0.001))])
text_clf.fit(train.textual_comb, train.label)  
label = text_clf.predict(test.textual_comb)
np.mean(label == test.label)  #0.6174445294567712，0.612088752869166

0.6013771996939556

In [0]:
from sklearn.preprocessing import OneHotEncoder
test['test']=label
one_hot = pd.get_dummies(test['content_type'])
one_hot2 = pd.get_dummies(test['test'])

test = test.drop('content_type',axis = 1)
test = test.drop('textual_comb',axis = 1)
test = test.drop('test',axis = 1)

test = test.join(one_hot)
test = test.join(one_hot2)

X=test.iloc[:,2:]
y=test.iloc[:,1]
test.head()



KeyError: ignored

In [0]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=6)
clf1 = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
clf1.fit(x_train, y_train)
y_pred=clf1.predict(x_test)
np.mean(y_pred == y_test)

0.5687022900763359

In [0]:
clf1.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)>

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

array([[112,   7,   1],
       [ 59,  17,   1],
       [ 44,   1,  20]])

In [0]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)



NameError: ignored

In [0]:
from sklearn.ensemble import RandomForestClassifier
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=6)
clf2 = RandomForestClassifier(n_estimators=50, max_depth=10)
clf2.fit(x_train, y_train)
y_pred=clf2.predict(x_test)
np.mean(y_pred == y_test)

0.5687022900763359

In [0]:
clf2.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [0]:
clf2.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [0]:
pipeline = Pipeline([('vec', CountVectorizer()),
                     ('tfidf', TfidfTransformer()), 
                     ('clf',MultinomialNB())])
parameters = {  
'vec__max_df': (0.1,0.25, 0.5, 0.75,1.0),  
'vec__min_df': (1, 5, 10, 20, 50),  
'vec__binary': (True, False),  
'vec__ngram_range':((1,1),(1,2),(1,3),(1,4)),
'tfidf__use_idf': (True, False),  
'tfidf__norm': ('l1', 'l2'),  
'clf__alpha': (1, 0.1, 0.01, 0.001, 0.0001)  
}  
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2)  
grid_search.fit(df_group.textual_comb,df_group.label)  
print("Best score: {0}".format(grid_search.best_score_))  
print("Best parameters set:")  
best_parameters = grid_search.best_estimator_.get_params()  
for param_name in sorted(list(parameters.keys())):  
    print("\t{0}: {1}".format(param_name, best_parameters[param_name]))
# https://stackoverflow.com/questions/26569478/performing-grid-search-on-sklearn-naive-bayes-multinomialnb-on-multi-core-machin

Fitting 3 folds for each of 4000 candidates, totalling 12000 fits
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 1) 
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 1) 
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 1), total=   5.6s
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 1), total=   5.8s
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 1) 
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_range=(1, 2) 
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=1, vec__ngram_

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 19.3min


[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 1), total=   6.6s
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 2) 
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=10, vec__ngram_range=(1, 4), total= 1.5min
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 2) 
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 2), total=  26.0s
[CV] clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 2) 
[CV]  clf__alpha=1, tfidf__norm=l1, tfidf__use_idf=True, vec__binary=True, vec__max_df=0.1, vec__min_df=20, vec__ngram_range=(1, 2), total=  24.7s
[CV] clf__alpha=